# Chapter 8. When Your App Grows

## 8.1 Introduction

(1) Grouwing applicatino

(2) Debugging

(3) Unit testing

(4) Blueprints

(5) Application factory

(6) Dynamic configuration

## 8.2 Demo: The Flask debug toolbar

(1) Install the Flask debuggin extension package `flask-debugtoolbar`

```bash
$ pip install flask-debugtoolbar
```

If you hit the error like "RuntimeError: flask-debugtoolbar signalling support is unavailable because the blinker library is not installed", then you also need to install the package `blinker`.

```
$ pip install blinker
```

(2) Enable `flask-debugtoolbar` in `__init__.py`.

In [ ]:
# SAVE AS __init__.py

# -*- coding: utf-8 -*-

import os

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager
from flask_moment import Moment
from flask_debugtoolbar import DebugToolbarExtension

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
# Need to set 'DEBUG` to True, otherwise the debug toolbar won't be shown.
app.config['DEBUG'] = True

db = SQLAlchemy(app)

# Configure authentication
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.login_view = 'login'
login_manager.init_app(app)

# Enable debugtoolbar
toolbar = DebugToolbarExtension(app)

# For displaying timestamps
moment = Moment(app)

from . import models
from . import views

(3) Play with the debug toolbar.

* Config
* Templates
* SQLAlchemy

Not covered in this course:

* Logging
* Profiler

Found many tag queries when `index.html` was rendered and fix it in `models.py`.

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

# Create a junction table which connects the foreign keys of two tables 
# Bookmark and Tag
tags = db.Table('bookmark_tag',
    db.Column('tag_id', db.Integer, db.ForeignKey('tag.id')),
    db.Column('bookmark_id', db.Integer, db.ForeignKey('bookmark.id')))

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    _tags = db.relationship('Tag', secondary=tags, lazy='joined',
                            backref=db.backref('bookmarks', lazy='dynamic'))
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
        
    @property
    def tags(self):
        return ",".join([t.name for t in self._tags])
        
    @tags.setter
    def tags(self, string):
        if string:
            self._tags = [Tag.get_or_create(name) for name in string.split(',')]
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username
        
class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(25), nullable=False, unique=True, index=True)
    
    @staticmethod
    def get_or_create(name):
        try:
            return Tag.query.filter_by(name=name).one()
        except:
            return Tag(name=name)
           
    @staticmethod
    def all():
        return Tag.query.all()
             
    def __repr__(self):
        return self.name

## 8.3 Demo: A blueprint

(1) Create a python sub-package for a blueprint `auth` which contains all the authentication-related views.

Notice that the route prefix in `views.py` is `auth` instead of `app`.

In [ ]:
# SAVE AS __init__.py of auth

# -*- coding: utf-8 -*-

from flask import Blueprint

auth = Blueprint('auth', __name__)

from . import views

In [ ]:
# SAVE AS views.py of auth

from flask import render_template, flash, redirect, url_for, request
from flask_login import login_user, logout_user

from . import auth
from .. import db
from ..models import User
from .forms import LoginForm, SignupForm

@auth.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # login and validate the user...
        user = User.get_by_username(form.username.data)
        if user is not None and user.check_password(form.password.data):
            login_user(user, form.remember_me.data)
            flash('Logged in successfully as {}'.format(user.username))
            return redirect(request.args.get('next') or url_for('user', username=user.username))
        flash('Incorrect username or password.')
    return render_template('login.html', form=form)
    
@auth.route('/logout')
def logout():
    logout_user()
    return redirect(url_for('index'))
    
@auth.route('/signup', methods=['GET', 'POST'])
def signup():
    form = SignupForm()
    if form.validate_on_submit():
        user = User(email=form.email.data,
                    username=form.username.data,
                    password=form.password.data)
        db.session.add(user)
        db.session.commit()
        flash('Welcome, {}! Please login'.format(user.username))
        return redirect(url_for('.login'))
    return render_template('signup.html', form=form)

In [ ]:
# SAVE AS forms.py of auth

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField, PasswordField, BooleanField, SubmitField
from wtforms.validators import DataRequired, Length, Email, Regexp, EqualTo,\
    ValidationError
    
class LoginForm(Form):
    username = StringField('Your Username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    remember_me = BooleanField('Keep me logged in')
    submit = SubmitField('Log In')
    
class SignupForm(Form):
    username = StringField('Username',
                    validators=[
                        DataRequired(), Length(3, 80),
                        Regexp('^[A-Za-z0-9_]{3,}$',
                            message='Usernames consist of numbers, letters, '
                                    'and underscores.')])
    password = PasswordField('Password',
                    validators=[
                        DataRequired(),
                        EqualTo('password2', message='Passwords must match')])
    password2 = PasswordField('Confirm password', validators=[DataRequired()])
    email = StringField('Email',
                        validators=[DataRequired(), Length(1, 80), Email()])
                        
    def validate_email(self, email_field):
        if User.query.filter_by(email=email_field.data).first():
            raise ValidationError('There already is a user with this email address.')
            
    def validate_username(self, username_field):
        if User.query.filter_by(username=username_field.data).first():
            raise ValidationError('This username is already taken.')

(2) Import the `auth` blueprint in `__init__.py` of thermos and use the blueprint in `views.py` and template htmls.

Notice that after we use the `auth` blueprint, all the authentication template pages will be prefixed by `/auth`, e.g., `/auth/login`.

In [ ]:
# SAVE AS __init__.py

# -*- coding: utf-8 -*-

import os

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager
from flask_moment import Moment
from flask_debugtoolbar import DebugToolbarExtension

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
# Need to set 'DEBUG` to True, otherwise the debug toolbar won't be shown.
app.config['DEBUG'] = True

db = SQLAlchemy(app)

# Configure authentication
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.login_view = 'auth.login'
login_manager.init_app(app)

# Enable debugtoolbar
toolbar = DebugToolbarExtension(app)

# For displaying timestamps
moment = Moment(app)

from .auth import auth as auth_blueprint
app.register_blueprint(auth_blueprint, url_prefix='/auth')

from . import models
from . import views

In [ ]:
# SAVE AS views.py

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request, abort
from flask_login import login_required, login_user, logout_user, current_user

from . import app, db, login_manager
from thermos.forms import BookmarkForm
from thermos.models import User, Bookmark, Tag
    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))

@app.route('/')
@app.route('/index')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           
@app.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        tags = form.tags.data
        bm = Bookmark(user=current_user, url=url, description=description, tags=tags)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('index'))
    return render_template('bookmark_form.html', form=form, title='Add a bookmark')
    
@app.route('/edit/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def edit_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    form = BookmarkForm(obj=bookmark)
    if form.validate_on_submit():
        form.populate_obj(bookmark)
        db.session.commit()
        flash("Stored '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    return render_template('bookmark_form.html', form=form, title='Edit bookmark')
    
@app.route('/delete/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def delete_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    if request.method == 'POST':
        db.session.delete(bookmark)
        db.session.commit()
        # BUGBUG: tags are not deleted if there are no more associated bookmarks.
        flash("Delete '{}'".format(bookmark.description))
        return redirect(url_for('user', username=current_user.username))
    else:
        flash('Please confirm deleting the bookmark.')
    return render_template('confirm_delete.html', bookmark=bookmark, nolinks=True)
  
@app.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
    
@app.route('/tag/<name>')
def tag(name):
    tag = Tag.query.filter_by(name=name).first_or_404()
    return render_template('tag.html', tag=tag)

@app.errorhandler(403)
def forbidden(e):
    return render_template('403.html'), 403
        
@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
@app.errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500
    
@app.context_processor
def inject_tags():
    return dict(all_tags=Tag.all)

## 8.4 Review: Blueprints

### 8.4.1 Blueprints

(1) A blueprint of how to construct or extend an application

* Not an app, but app-like
* Can provide views and/or resources like static files or templates, or models, etc.

(2) Purpose

* Break up growing application into modules
* Re-usable modules that can be registered on multiple apps

(3) Blueprints don't have to be in their own package

But most of the time, they will be

### 8.4.2 Creating and registering a blueprint

(1) Create a blueprint

```python
from flask import Blueprint
auth = Blueprint('auth', __name__)
from . import views
```

(2) Register a blueprint

```python
from .auth import auth as auth_blueprint
app.register_blueprint(auth_blueprint, url_prefix='/auth')
```

* The views of the blueprint are registered with `@auth.route` instead of `@app.route`.

* When referencing views from a blueprint (`url_for`):
    Prefix the name with the blueprint name: "`auth.login`"
    From within the blueprint itself: "`.login`"

## 8.5 Demo: More blueprints

(1) Create two more blueprints `bookmarks` and `main`.

(2) Register the blueprints in app's `__init__.py`.

In [ ]:
# SAVE AS __init__.py of bookmarks

# -*- coding: utf-8 -*-

from flask import Blueprint

bookmarks = Blueprint('bookmarks', __name__)

from . import views

In [ ]:
# SAVE AS forms.py of bookmarks

# -*- coding: utf-8 -*-

from flask_wtf import Form
from wtforms.fields import StringField
from wtforms.fields.html5 import URLField
from wtforms.validators import DataRequired, Regexp, url

class BookmarkForm(Form):
    url = URLField('The URL for your bookmark', validators=[DataRequired(), url()])
    description = StringField('Add an optional description:')
    tags = StringField('Tags', validators=[Regexp(r'^[a-zA-Z0-9, ]*$',
                    message='Tags can only contain letters and numbers')])
    
    def validate(self):
        if not (self.url.data.startswith("http://") or\
            self.url.data.startswith("https://")):
            self.url.data = "http://" + self.url.data
            
        if not Form.validate(self):
            return False
            
        if not self.description.data:
            self.description.data = self.url.data
            
        # filter out empty and duplicate tag names
        stripped = [t.strip() for t in self.tags.data.split(',')]
        not_empty = [tag for tag in stripped if tag]
        tagset = set(not_empty)
        self.tags.data = ','.join(tagset)
            
        return True

In [ ]:
# SAVE AS views.py of bookmarks

# -*- coding: utf-8 -*-

from flask import render_template, flash, redirect, url_for, request, abort
from flask_login import login_required, current_user

from . import bookmarks
from .forms import BookmarkForm
from .. import db
from ..models import User, Bookmark, Tag

@bookmarks.route('/add', methods=['GET', 'POST'])
@login_required
def add():
    form = BookmarkForm()
    if form.validate_on_submit():
        url = form.url.data
        description = form.description.data
        tags = form.tags.data
        bm = Bookmark(user=current_user, url=url, description=description, tags=tags)
        db.session.add(bm)
        db.session.commit()
        flash("Stored bookmark '{}' with description '{}'".format(url, description))
        return redirect(url_for('main.index'))
    return render_template('bookmark_form.html', form=form, title='Add a bookmark')
    
@bookmarks.route('/edit/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def edit_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    form = BookmarkForm(obj=bookmark)
    if form.validate_on_submit():
        form.populate_obj(bookmark)
        db.session.commit()
        flash("Stored '{}'".format(bookmark.description))
        return redirect(url_for('.user', username=current_user.username))
    return render_template('bookmark_form.html', form=form, title='Edit bookmark')
    
@bookmarks.route('/delete/<int:bookmark_id>', methods=['GET', 'POST'])
@login_required
def delete_bookmark(bookmark_id):
    bookmark = Bookmark.query.get_or_404(bookmark_id)
    if current_user != bookmark.user:
        abort(403)
    if request.method == 'POST':
        db.session.delete(bookmark)
        db.session.commit()
        # BUGBUG: tags are not deleted if there are no more associated bookmarks.
        flash("Delete '{}'".format(bookmark.description))
        return redirect(url_for('.user', username=current_user.username))
    else:
        flash('Please confirm deleting the bookmark.')
    return render_template('confirm_delete.html', bookmark=bookmark, nolinks=True)
  
@bookmarks.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
    
@bookmarks.route('/tag/<name>')
def tag(name):
    tag = Tag.query.filter_by(name=name).first_or_404()
    return render_template('tag.html', tag=tag)

In [ ]:
# SAVE AS __init__.py of main

# -*- coding: utf-8 -*-

from flask import Blueprint

main = Blueprint('main', __name__)

from . import views

In [ ]:
# SAVE AS views.py of main

# -*- coding: utf-8 -*-

from flask import render_template

from . import main
from .. import login_manager
from ..models import User, Bookmark, Tag

    
@login_manager.user_loader
def load_user(userid):
    return User.query.get(int(userid))


@main.route('/')
def index():
    return render_template('index.html', new_bookmarks=Bookmark.newest(5))
                           

@main.app_errorhandler(403)
def forbidden(e):
    return render_template('403.html'), 403
        
        
@main.app_errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404
    
    
@main.app_errorhandler(500)
def server_error(e):
    return render_template('500.html'), 500
    
    
@main.app_context_processor
def inject_tags():
    return dict(all_tags=Tag.all)

In [ ]:
# SAVE AS __init__.py of app
# Note that forms.py and views.py of app have broken down into blueprints, and they are deleted.

# -*- coding: utf-8 -*-

import os

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager
from flask_moment import Moment
from flask_debugtoolbar import DebugToolbarExtension

basedir = os.path.abspath(os.path.dirname(__file__))

app = Flask(__name__)
app.config['SECRET_KEY'] = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
# Need to set 'DEBUG` to True, otherwise the debug toolbar won't be shown.
app.config['DEBUG'] = True

db = SQLAlchemy(app)

# Configure authentication
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.login_view = 'auth.login'
login_manager.init_app(app)

# Enable debugtoolbar
toolbar = DebugToolbarExtension(app)

# For displaying timestamps
moment = Moment(app)

from .main import main as main_blueprint
app.register_blueprint(main_blueprint, url_prefix='/')

from .bookmarks import bookmarks as bkm_blueprint
app.register_blueprint(bkm_blueprint, url_prefix='/bookmarks')

from .auth import auth as auth_blueprint
app.register_blueprint(auth_blueprint, url_prefix='/auth')

## 8.6 Demo: An app factory and dynamic configuration

Use app factory for unit testing.

```bash
$ export THERMOS_ENV=dev
$ export THERMOS_ENV=test
$ export THERMOS_ENV=prod

$ python manager.py runserver
```

In [ ]:
## SAVE AS config.py

# -*- coding: utf-8 -*-

import os

basedir = os.path.abspath(os.path.dirname(__file__))

class Config:
    SECRET_KEY = b'c\x04\x14\x00;\xe44 \xf4\xf3-_9B\x1d\x15u\x02g\x1a\xcc\xd8\x04~'
    SQLALCHEMY_TRACK_MODIFICATIONS = False
    DEBUG = False
    
class DevelopmentConfig(Config):
    DEBUG = True
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'thermos.db')

class TestingConfig(Config):
    TESTING = True
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'data-test.db')
    WTF_CSRF_ENABLED = False
    SERVER_NAME = 'localhost'

class ProductionConfig(Config):
    DEBUG = False
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'thermos.db')
    
config_by_name = dict(
    dev = DevelopmentConfig,
    test = TestingConfig,
    prod = ProductionConfig
)

In [ ]:
## SAVE AS __init__.py

# -*- coding: utf-8 -*-

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager
from flask_moment import Moment
from flask_debugtoolbar import DebugToolbarExtension

from .config import config_by_name

db = SQLAlchemy()

# Configure authentication
login_manager = LoginManager()
login_manager.session_protection = 'strong'
login_manager.login_view = 'auth.login'

# Enable debugtoolbar
toolbar = DebugToolbarExtension()

# For displaying timestamps
moment = Moment()


def create_app(config_name):
    app = Flask(__name__)
    app.config.from_object(config_by_name[config_name])
    
    db.init_app(app)
    login_manager.init_app(app)
    moment.init_app(app)
    toolbar.init_app(app)

    from .main import main as main_blueprint
    app.register_blueprint(main_blueprint, url_prefix='/')

    from .bookmarks import bookmarks as bkm_blueprint
    app.register_blueprint(bkm_blueprint, url_prefix='/bookmarks')

    from .auth import auth as auth_blueprint
    app.register_blueprint(auth_blueprint, url_prefix='/auth')
    
    return app

In [ ]:
## SAVE AS manager.py

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os

from thermos import create_app, db
# Although Bookmark is not referenced in this file, it is needed to imported so that 
# it can be detected by the database migrator.
from thermos.models import User, Bookmark, Tag

from flask_script import Manager, prompt_bool
from flask_migrate import Migrate, MigrateCommand

app = create_app(os.getenv('THERMOS_ENV') or 'dev')
manager = Manager(app)

migrate = Migrate(app, db)
manager.add_command('db', MigrateCommand)
        
if __name__ == '__main__':
    manager.run()

## 8.7 Unit testing

(1) Install the flask test extension package `flask-testing` and the Python test tool package `nose`.

```bash
$ pip install flask-testing nose
```

(2) Create the test sub-package.

(3) Run the unit test.

```bash
$ nosetests
```

In [ ]:
# SAVE AS thermos_test.py

from flask import url_for
from flask.ext.testing import TestCase

import thermos
from thermos.models import User, Bookmark

class ThermosTestCase(TestCase):

    def create_app(self):
        return thermos.create_app('test')

    def setUp(self):
        self.db = thermos.db
        self.db.create_all()
        self.client = self.app.test_client()

        u = User(username='test', email='test@example.com', password='test')
        bm = Bookmark(user= u, url="http://www.example.com",
                      tags="one,two,three")
        self.db.session.add(u)
        self.db.session.add(bm)
        self.db.session.commit()

        self.client.post(url_for('auth.login'),
            data = dict(username='test', password='test'))

    def tearDown(self):
        thermos.db.session.remove()
        thermos.db.drop_all()

    def test_delete_all_tags(self):
        response = self.client.post(
            url_for('bookmarks.edit', bookmark_id=1),
            data = dict(
                url = "http://test.example.com",
                tags = ""
            ),
            follow_redirects = True
        )

        assert response.status_code == 200
        bm = Bookmark.query.first()
        assert not bm._tags

Find one bug in `models.py`.

In [ ]:
# SAVE AS models.py

# -*- coding: utf-8 -*-

from datetime import datetime

from sqlalchemy import desc
from flask_login import UserMixin
from werkzeug.security import check_password_hash, generate_password_hash

from . import db

# Create a junction table which connects the foreign keys of two tables 
# Bookmark and Tag
tags = db.Table('bookmark_tag',
    db.Column('tag_id', db.Integer, db.ForeignKey('tag.id')),
    db.Column('bookmark_id', db.Integer, db.ForeignKey('bookmark.id')))

class Bookmark(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    url = db.Column(db.Text, nullable=False)
    # Pass the function object instead of the function result 
    # as the default method for getting the default time.
    date = db.Column(db.DateTime, default=datetime.utcnow)  
    description = db.Column(db.String(300))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    _tags = db.relationship('Tag', secondary=tags, lazy='joined',
                            backref=db.backref('bookmarks', lazy='dynamic'))
    
    @staticmethod
    def newest(num):
        return Bookmark.query.order_by(desc(Bookmark.date)).limit(num)
        
    @property
    def tags(self):
        return ",".join([t.name for t in self._tags])
        
    @tags.setter
    def tags(self, string):
        if string:
            self._tags = [Tag.get_or_create(name) for name in string.split(',')]
        ## BUGBUG: The else statement was missing
        else: 
            self._tags = []
    
    def __repr__(self):
        return "<Bookmark '{}': '{}'>".format(self.description, self.url)

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)
    bookmarks = db.relationship('Bookmark', backref='user', lazy='dynamic')
    password_hash = db.Column(db.String)
    
    @property
    def password(self):
        raise AttributeError('password: write-only field')
        
    @password.setter
    def password(self, password):
        self.password_hash = generate_password_hash(password)
        
    def check_password(self, password):
        return check_password_hash(self.password_hash, password)
        
    @staticmethod
    def get_by_username(username):
        return User.query.filter_by(username=username).first()
    
    def __repr__(self):
        return '<User %r>' % self.username
        
class Tag(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(25), nullable=False, unique=True, index=True)
    
    @staticmethod
    def get_or_create(name):
        try:
            return Tag.query.filter_by(name=name).one()
        except:
            return Tag(name=name)
           
    @staticmethod
    def all():
        return Tag.query.all()
             
    def __repr__(self):
        return self.name

## 8.8 Resources and summary

### 8.8.1 Resources

(1) `flask-debugtoolbar`

https://github.com/mgood/flask-debugtoolbar

(2) `flask-testing`

https://pythonhosted.org/Flask-Testing/

(3) `nose`

https://nose.readthedocs.org/en/latest/

(4) `blueprints`

http://flask.pocoo.org/docs/latest/blueprints/

(5) Flask app factories

http://flask.pocoo.org/docs/latest/patterns/appfactories/

(6) Flask config

http://flask.pocoo.org/docs/latest/config/

### 8.8.2 Summary

(1) Flask debug toolbar

(2) Blueprints

(3) App factory

(4) Dynamic configuration

(5) Unit testing